In [4]:
import pandas as pd
import os
os.getcwd()

'/home/sagemaker-user/Code/NYC_Taxi_Prediction'

In [5]:
data = pd.read_parquet("/home/sagemaker-user/Code/NYC_Taxi_Prediction/data/train_data/")

In [6]:
data.shape

(144920, 15)

In [7]:
data.columns

Index(['trip_duration_mins', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'pulocationid', 'dolocationid', 'passenger_count', 'trip_distance',
       'vendorid', 'ratecodeid', 'day_of_week', 'day_of_month',
       'month_of_year', 'hour_of_day', 'week_of_year', 'features'],
      dtype='object')

In [18]:
data.describe()

,trip_duration_mins,tpep_pickup_datetime,tpep_dropoff_datetime,pulocationid,dolocationid,passenger_count,trip_distance,vendorid,ratecodeid,day_of_week,day_of_month,month_of_year,hour_of_day,week_of_year
count,144920.000000,144920,144920,144920.000000,144920.000000,144920.000000,144920.000000,144920.000000,144920.000000,144920.0,144920.0,144920.0,144920.000000,144920.0
mean,13.351028,2019-09-01 13:55:23.383956736,2019-09-01 14:08:44.445659648,155.566078,152.988476,1.670011,3.467067,1.663435,1.055707,1.0,1.0,9.0,13.420384,35.0
min,1.000000,2019-09-01 00:00:00,2019-09-01 00:01:06,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.0,1.0,9.0,0.000000,35.0
25%,6.367000,2019-09-01 10:12:49.750000128,2019-09-01 10:24:02.500000,107.000000,90.000000,1.000000,1.100000,1.000000,1.000000,1.0,1.0,9.0,10.000000,35.0
50%,10.617000,2019-09-01 14:55:43,2019-09-01 15:10:21,158.000000,158.000000,1.000000,1.900000,2.000000,1.000000,1.0,1.0,9.0,14.000000,35.0
75%,17.367000,2019-09-01 19:05:29,2019-09-01 19:19:52,230.000000,230.000000,2.000000,3.700000,2.000000,1.000000,1.0,1.0,9.0,19.000000,35.0
max,60.000000,2019-09-01 23:59:57,2019-09-02 00:50:22,265.000000,265.000000,6.000000,57.140000,2.000000,99.000000,1.0,1.0,9.0,23.000000,35.0
std,9.805103,NaN,NaN,66.827159,72.174834,1.256017,4.239497,0.472536,0.700990,0.0,0.0,0.0,6.572524,0.0


In [9]:
data.head()['features'][2]

{'type': 0,
 'size': 9332,
 'indices': array([30], dtype=int32),
 'values': array([1.])}

In [10]:
data.dtypes

trip_duration_mins              float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
pulocationid                      int32
dolocationid                      int32
passenger_count                   int32
trip_distance                   float64
vendorid                          int32
ratecodeid                        int32
day_of_week                       int32
day_of_month                      int32
month_of_year                     int32
hour_of_day                       int32
week_of_year                      int32
features                         object
dtype: object

In [124]:
from scipy.sparse import vstack, csr_matrix, hstack
import numpy as np

# Prints the available elements only of a CSR matrix
def print_csr_matrix(X):
    for i in range(X.shape[0]):
        start = X.indptr[i]
        end = X.indptr[i+1]
        cols = X.indices[start:end]
        vals = X.data[start:end]
        print(f"Row {i}: {list(zip(cols, vals))}")

# converts the sparse dictionary to csr matrix
def sparse_dict_to_csr(row):
    size = row['size']
    indices = row['indices']
    values = row['values']
    # row vector, shape=(1, size)
    return csr_matrix((values, (np.zeros_like(indices), indices)), shape=(1, size))

def get_scipy_sparse_csr_matrix(data, numeric_columns, onehot_vector_column='features'):
    '''
    Converts a pandas dataframe containing numerical columns and a single column that contains vector of one hot encoded dummy variables
    Return a scipy sparse csr matrix
    '''
    # First convert each value i.e. sparse dict to a sparse csr vertically stack them
    X_sparse = vstack([sparse_dict_to_csr(row) for row in data[onehot_vector_column]])
    # We need to convert all numeric features to sparse matrix too and horizontally stack with the above "features" sparse matrix 
    X_numeric = data[numeric_columns].to_numpy() # Dense matrix
    X_numeric_sparse = csr_matrix(X_numeric)   # Sparse matrix
    # concatenate the numeric and ohe columns into one numpy matrix
    X_all = hstack([X_numeric_sparse, X_sparse])
    print(X_all.shape)
    #print(X_all)
    #print_csr_matrix(X_all)
    return X_all

In [99]:
data2 = data#.sample(10000)
num_features = ["passenger_count", "trip_distance", "vendorid"]
X_numeric = data2[num_features].to_numpy() # First convert each value i.e. sparse dict to a sparse csr vertically stack them
X_sparse = vstack([sparse_dict_to_csr(row) for row in data2['features']])
#print(data2['features'])
#print_csr_matrix(X_sparse)
# We need to convert all numeric features to sparse matrix too and horizontally stack with the above "features" sparse matrix 
X_numeric_sparse = csr_matrix(X_numeric)
#print_csr_matrix(X_numeric_sparse)
X_all = hstack([X_numeric_sparse, X_sparse])
print(X_all.shape)
#print(X_all)
#print_csr_matrix(X_all)

(144920, 9335)


In [125]:
target_var = "trip_duration_mins"
features = "features|passenger_count|trip_distance|vendorid|ratecodeid|day_of_week|day_of_month|month_of_year|hour_of_day|week_of_year" 
features = features.split('|')

In [126]:
train = data.sample(10)
train_target = train[target_var]
train = train[features]
    
list(train.columns)
list(train.dtypes)
numeric_columns = train.select_dtypes(include=["number"]).columns.tolist()
print(numeric_columns)


['passenger_count', 'trip_distance', 'vendorid', 'ratecodeid', 'day_of_week', 'day_of_month', 'month_of_year', 'hour_of_day', 'week_of_year']


In [127]:
if 'features' in features: # if one hot encode feature vectors is saved under column "features"
    train_np = get_scipy_sparse_csr_matrix(train, numeric_columns, onehot_vector_column='features')
else:
    train_np = train[features].values

(10, 9341)


In [128]:
print(train_np)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 99 stored elements and shape (10, 9341)>
  Coords	Values
  (0, 0)	1.0
  (0, 1)	0.99
  (0, 2)	2.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	9.0
  (0, 7)	3.0
  (0, 8)	35.0
  (0, 824)	1.0
  (1, 0)	1.0
  (1, 1)	3.0
  (1, 2)	2.0
  (1, 3)	1.0
  (1, 4)	1.0
  (1, 5)	1.0
  (1, 6)	9.0
  (1, 7)	14.0
  (1, 8)	35.0
  (1, 1061)	1.0
  (2, 0)	1.0
  (2, 1)	3.19
  (2, 2)	2.0
  (2, 3)	1.0
  (2, 4)	1.0
  :	:
  (7, 4)	1.0
  (7, 5)	1.0
  (7, 6)	9.0
  (7, 8)	35.0
  (7, 1952)	1.0
  (8, 0)	1.0
  (8, 1)	10.82
  (8, 2)	2.0
  (8, 3)	1.0
  (8, 4)	1.0
  (8, 5)	1.0
  (8, 6)	9.0
  (8, 7)	7.0
  (8, 8)	35.0
  (8, 1371)	1.0
  (9, 0)	1.0
  (9, 1)	14.59
  (9, 2)	2.0
  (9, 3)	1.0
  (9, 4)	1.0
  (9, 5)	1.0
  (9, 6)	9.0
  (9, 7)	12.0
  (9, 8)	35.0
  (9, 2065)	1.0


In [3]:
import pandas as pd
data  = pd.read_parquet("/home/sagemaker-user/part-00717-a2fc98e8-9f09-4957-9e18-33363f703c76.c000.snappy.parquet")

In [5]:
data.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,cbd_congestion_fee
0,2,2024-03-07 00:00:25,2024-03-07 00:12:00,1,4.94,1,N,132,219,1,21.2,1.0,0.5,0.00,0.00,1.0,25.45,0.0,1.75,NaN
1,2,2024-03-07 00:06:11,2024-03-07 00:18:16,1,1.58,1,N,230,170,1,12.1,1.0,0.5,1.71,0.00,1.0,18.81,2.5,0.00,NaN
2,1,2024-03-07 00:32:39,2024-03-07 00:40:04,1,0.40,1,N,140,141,2,5.8,3.5,0.5,0.00,0.00,1.0,10.80,2.5,0.00,NaN
3,1,2024-03-07 00:09:09,2024-03-07 00:16:58,0,1.30,1,N,246,158,1,10.0,3.5,0.5,3.00,0.00,1.0,18.00,2.5,0.00,NaN
4,2,2024-03-07 00:06:41,2024-03-07 00:24:36,1,10.67,1,N,129,47,2,40.8,1.0,0.5,0.00,6.94,1.0,50.24,0.0,0.00,NaN


In [9]:
data_path = "/home/sagemaker-user/Code/NYC_Taxi_Prediction/part-00000-56406aea-6a52-4e8a-89b7-f56010e3948c-c000.snappy.parquet"

In [11]:
import pandas as pd   
data = pd.read_parquet(data_path)
data.head()

,trip_duration_mins,tpep_pickup_datetime,tpep_dropoff_datetime,pulocationid,dolocationid,passenger_count,trip_distance,vendorid,ratecodeid,day_of_week,day_of_month,month_of_year,hour_of_day,week_of_year,features
0,11.933,2024-03-14 19:32:33,2024-03-14 19:44:29,164,142,1,2.00,1,1,5,14,3,19,11,"{'type': 0, 'size': 40998, 'indices': [714], '..."
1,56.400,2024-02-24 22:26:34,2024-02-24 23:22:58,132,246,2,17.60,1,2,7,24,2,22,8,"{'type': 0, 'size': 40998, 'indices': [1338], ..."
2,18.800,2024-03-14 18:34:07,2024-03-14 18:52:55,236,164,1,2.33,2,1,5,14,3,18,11,"{'type': 0, 'size': 40998, 'indices': [697], '..."
3,15.967,2024-03-14 19:59:02,2024-03-14 20:15:00,141,151,1,3.20,1,1,5,14,3,19,11,"{'type': 0, 'size': 40998, 'indices': [1611], ..."
4,20.783,2024-02-29 00:12:03,2024-02-29 00:32:50,186,74,2,5.07,2,1,5,29,2,0,9,"{'type': 0, 'size': 40998, 'indices': [2076], ..."


In [46]:
data['pickup_drop_location'] = data['pulocationid'].astype('str') +'-'+ data['dolocationid'].astype('str')
data['pickup_drop_location'] = data['pickup_drop_location'].astype('category')
data['pickup_drop_location'].value_counts()
print(data['pickup_drop_location'].nunique())

5667


In [33]:
data.sample(frac=.5).shape

(33190, 16)

In [30]:
data.shape

(66379, 16)

In [34]:
temp = data['pickup_drop_location'].value_counts()

In [48]:
rare_trips = list(temp[temp < 20].index)
import numpy as np
data['pickup_drop_location'] = np.where(data['pickup_drop_location'].isin(rare_trips), 'Other',data['pickup_drop_location'])

In [50]:
data['pickup_drop_location'].value_counts()

pickup_drop_location
Other      19127
237-236      479
236-237      439
236-236      332
237-237      326
           ...  
48-100        20
239-186       20
164-211       20
141-164       20
263-143       20
Name: count, Length: 935, dtype: int64